In [25]:
import pandas as pd
import seaborn as sns
import numpy as np


In [16]:

df_train = pd.read_csv("C:/Users/Spurthi Bollina/Documents/Titanic Sruvival Challenge/train.csv")
df_test = pd.read_csv("C:/Users/Spurthi Bollina/Documents/Titanic Sruvival Challenge/test.csv")

df_train.describe() # Summary on numeric values
df_train.isnull().sum() # Age, Cabin and Embarked has missing values
df_test.isnull().sum() # Age, Fare and Cabin has missing values
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [43]:
from sklearn import preprocessing
def encode_features(df_train, df_test):
    features = ['Sex']
    df_combined = pd.concat([df_train[features], df_test[features]])
    
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df_combined[feature])
        df_train[feature] = le.transform(df_train[feature])
        df_test[feature] = le.transform(df_test[feature])
    return df_train, df_test
    
data_train, data_test = encode_features(df_train, df_test)
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,90,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,359,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,156,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,339,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,161,NaN,S


In [45]:

# Modelling without any pre-processing, excluding unique fields

from sklearn.model_selection import train_test_split

# Removing missing and categorical variables
train_x = df_train.drop(['Name', 'PassengerId', 'Survived', 'Ticket', 'Age', 'Cabin', 'Fare', 'Embarked'], axis = 1) 
train_y = df_train['Survived']

# splitting train data as train and validation
num_test = 0.20 
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size = num_test, random_state = 23)



,Pclass,Sex,SibSp,Parch
151,1,0,1,0
753,3,1,0,0
746,3,1,1,1
684,2,1,1,1
887,1,0,0,0


In [50]:
# RondomForest 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

# Choose the type of classifier. 
clf = RandomForestClassifier()

# Choose some parameter combinations to try
parameters = {'n_estimators': [4, 6, 9],# no of trees in the forest
              'max_features': ['log2', 'sqrt','auto'],  #The number of features to consider when looking for the best split:
              'criterion': ['entropy', 'gini'], # The function to measure the quality of a split
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5], #The minimum number of samples required to split an internal node
              'min_samples_leaf': [1,5,8] # The minimum number of samples required to be at a leaf node
             }


# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(x_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
clf.fit(x_train, y_train)





RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=8, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=6, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [52]:
predictions = clf.predict(x_test)
print(accuracy_score(y_test, predictions))

0.793296089385


In [54]:
# Kfold Cross Validation

from sklearn.cross_validation import KFold

def run_kfold(clf, X_all, y_all):
    kf = KFold(891, n_folds=10)
    outcomes = []
    fold = 0
    for train_index, test_index in kf:
        fold += 1
        X_train, X_test = X_all.values[train_index], X_all.values[test_index]
        y_train, y_test = y_all.values[train_index], y_all.values[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Fold {0} accuracy: {1}".format(fold, accuracy))     
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 
    
run_kfold(clf, train_x, train_y)

Fold 1 accuracy: 0.7333333333333333
Fold 2 accuracy: 0.8202247191011236
Fold 3 accuracy: 0.7303370786516854
Fold 4 accuracy: 0.797752808988764
Fold 5 accuracy: 0.8089887640449438
Fold 6 accuracy: 0.7528089887640449
Fold 7 accuracy: 0.8089887640449438
Fold 8 accuracy: 0.7191011235955056
Fold 9 accuracy: 0.8651685393258427
Fold 10 accuracy: 0.7865168539325843
Mean Accuracy: 0.7823220973782771


In [64]:
ids = df_test['PassengerId']
test=df_test[['Pclass', 'Sex', 'SibSp', 'Parch']]
test.head()
predictions = clf.predict(test)
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [66]:
pip install xgboost


The following command must be run outside of the IPython shell:

    $ pip install xgboost

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [1]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(x_train, y_train)
predictions = gbm.predict(test_X)

ModuleNotFoundError: No module named 'xgboost'

'C:\\Anaconda'